# RL Exercise 6 - Training with Ray and Serving with Clipper

**GOAL:** The goal of this exercise is to show how to train a policy with Ray and to deploy it with Clipper in a fun, interactive way.

We will train an agent to play Pong, and then we will play Pong against the policy that we trained.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import pong_py
import ray

from ray.tune.registry import register_env
from ray.rllib.agents import ppo
import flor

In [2]:
flor.setNotebookName('flor_rl_exercise04.ipynb')

In [3]:
ex = flor.Experiment('integration_risecamp2018').__enter__()

In [4]:
imitation_data = ex.artifact('imitation_data.csv', 'imitation_data')

In [5]:
@flor.func
def preproc_imitation(imitation_data, procd_imitation_data, **kwargs):
    import pandas as pd
    df_data = pd.read_csv(imitation_data)
    df_data.columns = ["label","paddle_y","ball_x","ball_y","ball_dx","ball_dy","x_prev","y_prev"]

    def convert_label(label):
        """Convert labels into numeric values"""
        if(label=="down"):
            return 1
        elif(label=="up"):
            return 2
        else:
            return 0

    df_data['label'] = df_data['label'].apply(convert_label)
    df_data.loc[:, "paddle_y":"y_prev"] = df_data.loc[:, "paddle_y":"y_prev"]/500.0
    df_data.to_json(procd_imitation_data)

In [6]:
do_preproc_imitation = ex.action(preproc_imitation, [imitation_data])

In [7]:
procd_imitation_data = ex.artifact('imitation_data.json', 'procd_imitation_data', do_preproc_imitation)

In [8]:
@flor.func
def train_imitation_model(procd_imitation_data, imitation_model, **kwargs):
    import cloudpickle
    import pandas as pd
    from sklearn import linear_model
    df_data = pd.read_json(procd_imitation_data)
    
    labels = df_data['label']
    training_data= df_data.drop(['label'], axis=1)

    model = linear_model.LogisticRegression()
    model.fit(training_data, labels)
    with open(imitation_model, 'wb') as f:
        cloudpickle.dump(model, f)

    

In [9]:
do_train_imitation_model = ex.action(train_imitation_model, [procd_imitation_data])

In [10]:
imitation_model = ex.artifact('model.pkl', 'imitation_model', do_train_imitation_model)

In [11]:
ex.__exit__()

In [ ]:
imitation_model.plot()

In [12]:
imitation_model.pull()

Start Ray.

In [ ]:
@flor.func
def start_ray():
    try:
        ray.get([])
    except:
        ray.init()    
    return 0

Instantiate an agent that can be trained using Proximal Policy Optimization (PPO).

In [ ]:
@flor.func
def instantiate_agent(env_config, **kwargs):
    register_env("my_env", lambda ec: pong_py.PongJSEnv())
    trainer = ppo.PPOAgent(env="my_env", config={"env_config": {}})
    return trainer

Train the `PPOAgent` for some number of iterations.

**EXERCISE:** You will need to experiment with the number of iterations as well as with the configuration to get the agent to learn something reasonable.

In [ ]:
for i in range(2):
    result = trainer.train()

Use the agent manually by calling `agent.compute_action` and see the rewards you get are consistent with the rewards printed during the training procedure.

In [ ]:
env = pong_py.PongJSEnv()

for _ in range(20):
    state = env.reset()
    done = False
    cumulative_reward = 0

    while not done:
        action = trainer.compute_action(state)
        state, reward, done, _ = env.step(action)
        cumulative_reward += reward

    print(cumulative_reward)

Checkpoint the agent so that the relevant model can be saved and deployed to Clipper. We save the name of the checkpoint file in `metadata.json` so the model container knows how to restore the policy checkpoint.

In [ ]:
import os
import json
checkpoint_path = trainer.save()
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_file = os.path.basename(checkpoint_path)
with open(os.path.join(checkpoint_dir, "metadata.json"), "w") as f:
    json.dump({"checkpoint": checkpoint_file}, f)

## Play Against the Policy

In this section, we will play Pong against the policy that we just trained. The game will be played in your browser, and the policy that we trained will be served by Clipper.

**EXERCISE:** Deploy your policy using Clipper. Follow the instructions that get printed below to play Pong against the deployed policy. You'll need to deploy all of the data that is saved in the directory `os.path.dirname(checkpoint_path)`.

Start by importing the `clipper_admin` library and use that to create a new Clipper instance to serve the policy.

When you create your ClipperConnection, you need to tell it how to communicate with the Docker service and Clipper. You can use the following command to get the Docker IP address. Use that address when you create your `ClipperConnection` in the next step.

In [ ]:
# Make logging work correctly in the Jupyter notebook
import logging
import sys
import subprocess32 as subprocess
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from clipper_admin import DockerContainerManager, ClipperConnection
docker_ip = subprocess.check_output("./get_docker_ip.sh").strip()
clipper_conn = ClipperConnection(DockerContainerManager(docker_ip_address=docker_ip))
# Add a call to stop all in case you still have Clipper running from the earlier exercises
clipper_conn.stop_all()
clipper_conn.start_clipper()

Next, deploy the saved policy checkpoint to Clipper using a Docker image we created for this exercise (similar to the TensorFlow model container in the Clipper tutorial). If you're curious, you can find the custom model container code on [GitHub](https://github.com/ucbrise/risecamp/blob/077aa51078e2043d4d3d2d539e256c30c259678e/rl_and_pong/pong_model_container.py).

In [ ]:
import os
model_name = "pong-policy"
app_name = "pong"
clipper_conn.build_and_deploy_model(
    name=model_name,
    version=1,
    input_type="doubles",
    model_data_path=os.path.dirname(checkpoint_path),
    base_image="clipper/risecamp-pong-container"
)

Finally, register a Clipper application and link it the deployed policy model.

In [ ]:
app_name = "pong"
clipper_conn.register_application(name=app_name, default_output="0", input_type="doubles", slo_micros=100000)
clipper_conn.link_model_to_app(app_name=app_name, model_name=model_name)

Now that you have deployed your policy to Clipper, you will start a Pong application that will let you play against your policy in the browser.

When you start the application, you need to tell it where Clipper is running in order for the Pong application to request predictions from Clipper. `ClipperConnection` provides the `get_query_addr()` method to get the IP address and port on which Clipper is listening for incoming prediction requests.

In [ ]:
clipper_addr = clipper_conn.get_query_addr()
print("Clipper address: {}".format(clipper_addr))

Now you can start the Pong webserver. It will print out the URL it's running on after it starts. Copy and paste that URL into your browser and press "1" to play against your policy!

In [ ]:
import subprocess32 as subprocess
server_handle = subprocess.Popen(["./start_webserver.sh", clipper_addr], stdout=subprocess.PIPE)
print(server_handle.stdout.readline().strip())

## Deploy a New Policy

The first policy that you deploy probably won't be a very strong competitor, especially if you only trained it for a few iterations. Try training it for more iterations and deploying the new policy to Clipper. Clipper will automatically switch the Pong application to query the new version of the policy. You don't need to reload the page or even restart the game.

For your convenience, we've copied the relevant cells from above to train the policy for more iterations and deploy it Clipper. You can run this cell as many times as you want. Don't forget to increment the version number of the model each time you deploy to Clipper.

In [ ]:
# Train for more iterations
for i in range(50):
    result = agent.train()
    
# Save the new policy
checkpoint_path = agent.save()
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_file = os.path.basename(checkpoint_path)
with open(os.path.join(checkpoint_dir, "metadata.json"), "w") as f:
    json.dump({"checkpoint": checkpoint_file}, f)
    
# Deploy the new policy to Clipper.
clipper_conn.build_and_deploy_model(
    name=model_name,
    version=2, # If you run this more than once, don't forget to keep updating the version.
    input_type="doubles",
    model_data_path=os.path.dirname(checkpoint_path),
    base_image="clipper/risecamp-pong-container"
)